In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import GPT2Model, GPT2Config, PreTrainedModel
from sru import SRUpp
from datasets import load_dataset, Dataset, DatasetDict

from torch.utils.data import DataLoader
from transformers import Trainer, TrainingArguments

import math
import numpy as np

# Competitive edge techniques:
- Custom loss function based on profit with trade sizing (more of a conceptual innovation)
- multiloss to integrate shorter and longer trades (practical innovation to unlock longer times and more data and better gradients)
- best in class transformer -- most people don't really understand how transformers even work lol
    - hopefully will get more data efficient models
- the way I normalize the data may be better too    

- right now model is quite greedy (for my own sanity), but it's totally possible to consider more of a <50% accuracy model but just with higher upside

TODO:
- compute metrics functions
    - still need say a below <30min full trade %, accuracy profitability and over 30+ full trade % and accuracy and profitability
- feel like the input layer norm should be done away with...
- two major questions:
    - are my number correct and not cheated?
    - robustness about if results are feasibile in practice
- data analysis on when/where the model trades (which time frame and what time period throughout day)
    - should I do a graphical analysis?
    - what about raw accuracy of big trades?
- trickier goal, do usd/jpy and usd/gbp, should they be integrated all at once though? (would be tricky to handle multiple datastreams) may leave this for later after setting up paper trading
    - would be fine to just use it as a transfer learning tool though

In [2]:
class SRUTrader(PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        
        config.initializer_range = 1 / math.sqrt(config.n_embd)
        
        self.embed = nn.Linear(5, config.n_embd, bias = False)
        self.position_embeddings = nn.Embedding(config.n_positions, config.n_embd)
        self.norm = nn.LayerNorm(config.n_embd)
        self.sru = SRUpp(input_size = config.n_embd,
                         hidden_size = config.n_embd,
                         proj_size = 4 * config.n_embd, # paper says 8 is better, but working with memory contraints
                         num_layers = 10, # paper seemed to have tuned to find this to work
                         dropout = .01,
                         attn_dropout = .01,
                         rescale = True,
                         layer_norm = True,
                         attention_every_n_layers = 2)
        self.trade = nn.Linear(config.n_embd, 120, bias = False)


    def forward(self, ohlcv, future):
        # manual positional embeddings
        batch_size, seq_length, _ = ohlcv.shape
        position_ids = torch.arange(seq_length, dtype=torch.long, device=ohlcv.device)
        position_ids = position_ids.unsqueeze(0).expand(batch_size, -1)
        position_embeddings = self.position_embeddings(position_ids)
        
        embed = self.norm(self.embed(ohlcv) + position_embeddings)
        embed = torch.permute(embed, (1, 0, 2)) # sequence first for SRU
        hidden = torch.permute(self.sru(embed)[0], (1, 0, 2))
        
        soft_trade = self.trade(hidden)
        
        # sharpe information
        soft_trade = torch.tanh(soft_trade)
        soft_profit = soft_trade * future
        
        # the exp is so that loss is purely positive and minimizes toward 0 (also losses have more loss than profit)
        loss_ppl = torch.square(((-soft_profit + future.abs()))).mean()
        
        # penalty for big trades (to stop trading from happening with no profit)
        trade_penalty = soft_trade.abs().mean()
        
        loss = loss_ppl + .1 * trade_penalty # .1 means that a 100% position must make at least .1 of a std to offset loss
        
        return {
            'loss': loss,
            'profits': soft_profit,
            'trades': soft_trade,
        }

In [4]:
def compute_metrics(preds):
    soft_profit, soft_trade = preds.predictions
    abs_trade = np.abs(soft_trade)
    trades = abs_trade.sum()
    
    day_profits = soft_profit.sum(axis = (1, 2))
    
    metrics = {
        'day sharpe': day_profits.mean() / day_profits.std(),
        'trade %': trades * 100 / soft_profit.size,
        'full trade %': (abs_trade > .9).mean() * 100,
        'full trade accuracy': (soft_profit[abs_trade > .9] > 0).mean() * 100,
        'medium trade %': (abs_trade > .5).mean() * 100,
        'medium trade accuracy': (soft_profit[abs_trade > .5] > 0).mean() * 100,
    }
    
    # round the metrics
    metrics = {k: np.format_float_positional(v, precision = 2) for k, v in metrics.items()}
    
    return metrics

In [5]:
eurusd = Dataset.load_from_disk('data/EURUSD_day.ds')

# make splits
split = eurusd.train_test_split(.05, shuffle = False)
valid_test = split['test'].train_test_split(.5, shuffle = False)
eurusd = DatasetDict({
    'train': split['train'],
    'validation': valid_test['train'],
    'test': valid_test['test']
})

In [6]:
training_args = TrainingArguments(
    output_dir = "./results",
    logging_strategy = "steps",
    evaluation_strategy = "steps",
    logging_steps = 100,
    eval_steps = 100,
    report_to = "none",
    learning_rate = 5e-4,
    lr_scheduler_type = "cosine",
    warmup_ratio = .05,
    num_train_epochs = 1,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    max_grad_norm = 1,
)

In [13]:
config = GPT2Config(
    n_embd = 384, n_positions = 2000
)

In [14]:
model = SRUTrader(config)
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = eurusd['train'],
    eval_dataset = eurusd['validation'],
    compute_metrics = compute_metrics
)

In [9]:
# sru
trainer.train()

C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3270
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 3270
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Day sharpe,Trade %,Full trade %,Full trade accuracy,Medium trade %,Medium trade accuracy
100,0.707000,0.511279,1.23,37.33,4.36,66.84,30.01,56.53
200,1.117200,0.573373,1.47,58.55,24.83,54.93,59.16,52.02
300,0.452700,0.496984,2.11,49.29,18.54,60.35,45.26,56.11
400,0.537700,0.499412,1.34,56.95,26.6,61.27,55.62,58.81
500,0.514200,0.305154,1.43,31.76,5.19,80.69,20.34,68.63
600,0.388700,0.331800,1.54,27.54,6.15,78.10,13.94,70.29
700,0.476200,0.404420,1.88,59.19,18.03,67.74,62.29,61.70
800,0.422000,0.292924,1.46,32.75,4.56,84.47,21.5,72.27
900,0.407600,0.298788,1.46,35.89,4.53,83.85,26.33,69.70
1000,0.493400,0.427605,1.49,30.62,8.22,61.58,22.05,63.34


***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
Saving model checkpoint to ./results\checkpoint-500
Configuration saved in ./results\checkpoint-500\config.json
Model weights saved in ./results\checkpoint-500\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
Saving model checkpoint to ./results\checkpoint-1000
Configuration saved in ./results\checkpoint-1000\config.json
Model weights saved in ./res

TrainOutput(global_step=3270, training_loss=0.4054160027693536, metrics={'train_runtime': 1613.905, 'train_samples_per_second': 2.026, 'train_steps_per_second': 2.026, 'total_flos': 0.0, 'train_loss': 0.4054160027693536, 'epoch': 1.0})

In [10]:
trainer.evaluate(eurusd['test'])

***** Running Evaluation *****
  Num examples = 87
  Batch size = 1


{'eval_loss': 0.23087768256664276,
 'eval_day sharpe': '2.23',
 'eval_trade %': '39.68',
 'eval_full trade %': '8.79',
 'eval_full trade accuracy': '93.71',
 'eval_medium trade %': '34.06',
 'eval_medium trade accuracy': '82.64',
 'eval_runtime': 14.2324,
 'eval_samples_per_second': 6.113,
 'eval_steps_per_second': 6.113,
 'epoch': 1.0}

In [ ]:
del trainer
del model
torch.cuda.empty_cache()

# Appendix

## gpt2 experiements

In [ ]:
class GPT2Trader(PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        
        # use levine 2020 layer numbers
        n_layer = round((math.log(config.n_embd) - 5.039) / 5.55e-2)
        n_layer = max(1, n_layer)
        print(f'Using {n_layer} layers')
        config.n_layer = n_layer
        
        config.initializer_range = 1 / math.sqrt(config.n_embd)
        
        self.embed = nn.Linear(5, config.n_embd, bias = False)
        self.norm = nn.LayerNorm(config.n_embd)
        self.gpt = GPT2Model(config)
        self.trade = nn.Linear(config.n_embd, 120, bias = False)


    def forward(self, ohlcv, future):
        embed = self.norm(self.embed(ohlcv))
        hidden = self.gpt(inputs_embeds = embed).last_hidden_state
        
        soft_trade = self.trade(hidden)
        
        # sharpe information
        soft_trade = torch.tanh(soft_trade)
        soft_profit = soft_trade * future
        
        # the exp is so that loss is purely positive and minimizes toward 0 (also losses have more loss than profit)
        loss_ppl = torch.square(((-soft_profit + future.abs()))).mean()
        
        # penalty for big trades (to stop trading from happening with no profit)
        trade_penalty = soft_trade.abs().mean()
        
        loss = loss_ppl + .1 * trade_penalty # .1 means that a 100% position must make at least .1 of a std to offset loss
        
        return {
            'loss': loss,
            'profits': soft_profit,
            'trades': soft_trade,
        }

In [ ]:
config = GPT2Config(
    n_embd = 384, n_head = 6, vocab_size = 0, n_positions = 2000,
    resid_pdrop = .01, embd_pdrop = .01, attn_pdrop = .01, # low dropout since only using 1 epoch training and to make model more robust to data issues (.1 has worse loss, accuracy & t-score)
    summary_first_dropout = 0, summary_proj_to_labels = False,
    scale_attn_by_inverse_layer_idx = True, use_cache = False
)
model = GPT2Trader(config)

In [52]:
# 5e-4 lr (best)
trainer.train()

C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3270
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 3270
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Day sharpe,Trade %,Full trade %,Full trade accuracy,Medium trade %,Medium trade accuracy
100,0.739100,0.576555,1.18,55.53,0.,nan,68.10,52.68
200,1.038700,0.508294,1.57,25.15,0.04,53.85,19.25,56.31
300,0.546400,0.482877,1.60,33.05,0.02,43.69,32.39,53.77
400,0.691100,0.527884,1.36,42.9,1.1,61.84,34.89,54.52
500,0.639200,0.453005,1.28,27.61,0.,nan,19.7,57.28
600,0.492500,0.488201,1.82,38.68,0.,nan,34.53,54.87
700,0.654900,0.472404,1.43,36.18,0.00,50.99,31.99,54.76
800,0.555600,0.428319,1.41,21.04,0.00,42.86,17.26,58.88
900,0.554300,0.445764,1.26,25.83,0.16,52.64,15.78,59.76
1000,0.662100,0.395642,1.37,19.72,4.11,77.08,8.08,68.22


***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
<ipython-input-43-52e946b60fdf>:12: RuntimeWarning: Mean of empty slice.
  'full trade accuracy': (soft_profit[abs_trade > .9] > 0).mean() * 100,
C:\Users\micha\anaconda3\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
Saving model checkpoint to ./results\checkpoint-500
Configuration saved in ./results\checkpoint-500\config.json
Model weights saved in ./results\checkpoint-500\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
<ipython-input-43-52e946b60fdf>:12: RuntimeWarning: Mean of empty slice.
  'full trade accuracy': (s

TrainOutput(global_step=3270, training_loss=0.5385216050920866, metrics={'train_runtime': 1883.0698, 'train_samples_per_second': 1.737, 'train_steps_per_second': 1.737, 'total_flos': 0.0, 'train_loss': 0.5385216050920866, 'epoch': 1.0})

In [48]:
# 1e-3 lr
trainer.train()

C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3270
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 3270
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Day sharpe,Trade %,Full trade %,Full trade accuracy,Medium trade %,Medium trade accuracy
100,0.742800,0.598860,1.4,53.69,0.34,52.07,58.1,52.67
200,1.059300,0.629185,1.19,35.43,0.05,50.63,19.08,53.12
300,0.618700,0.574223,1.69,58.2,0.1,55.6,75.46,51.46
400,0.724900,0.498308,1.4,30.9,0.00,50.,29.13,53.94
500,0.643000,0.475523,1.06,39.59,0.14,46.61,24.92,55.38
600,0.504500,0.475192,1.70,38.56,0.06,56.23,36.72,54.34
700,0.706500,0.509966,1.87,37.81,0.,nan,42.78,52.66
800,0.584600,0.460770,1.35,21.82,0.88,50.54,17.30,56.59
900,0.572900,0.467708,1.16,24.87,0.00,100.,18.21,57.44
1000,0.735000,0.469905,1.38,25.93,4.13,63.69,17.26,58.26


***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
Saving model checkpoint to ./results\checkpoint-500
Configuration saved in ./results\checkpoint-500\config.json
Model weights saved in ./results\checkpoint-500\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
<ipython-input-43-52e946b60fdf>:12: RuntimeWarning: Mean of empty slice.
  'full trade accuracy': (soft_profit[abs_trade > .9] > 0).mean() * 100,
C:\Users\micha\anaconda3\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
***** Running Evaluation *****
 

TrainOutput(global_step=3270, training_loss=0.5552892037487905, metrics={'train_runtime': 1886.12, 'train_samples_per_second': 1.734, 'train_steps_per_second': 1.734, 'total_flos': 0.0, 'train_loss': 0.5552892037487905, 'epoch': 1.0})

In [56]:
# 2e-4 lr
trainer.train()

C:\Users\micha\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3270
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 3270
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Day sharpe,Trade %,Full trade %,Full trade accuracy,Medium trade %,Medium trade accuracy
100,0.740600,0.527701,2.02,38.95,0.09,52.46,32.52,53.95
200,0.977000,0.492953,1.67,26.88,0.09,59.97,21.80,56.01
300,0.539400,0.543064,1.8,29.15,0.00,100.,27.00,52.78
400,0.670100,0.463039,1.55,29.99,0.00,20.,27.,55.51
500,0.625400,0.451693,1.31,31.58,0.00,0.,25.62,55.81
600,0.480400,0.496943,1.92,42.34,0.,nan,41.84,53.82
700,0.660500,0.484283,1.5,28.90,0.01,51.19,26.33,56.26
800,0.563100,0.441980,1.36,22.58,0.,nan,20.,56.21
900,0.565500,0.439754,1.29,24.82,0.00,50.,18.22,57.8
1000,0.710500,0.449362,1.41,22.11,0.04,65.72,17.88,58.45


***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
Saving model checkpoint to ./results\checkpoint-500
Configuration saved in ./results\checkpoint-500\config.json
Model weights saved in ./results\checkpoint-500\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
<ipython-input-43-52e946b60fdf>:12: RuntimeWarning: Mean of empty slice.
  'full trade accuracy': (soft_profit[abs_trade > .9] > 0).mean() * 100,
C:\Users\micha\anaconda3\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
***** Running Evaluation *****
  Num examples = 86
  Batch size = 1
***** Running Evaluation *****
 

TrainOutput(global_step=3270, training_loss=0.5430762673007603, metrics={'train_runtime': 1879.3917, 'train_samples_per_second': 1.74, 'train_steps_per_second': 1.74, 'total_flos': 0.0, 'train_loss': 0.5430762673007603, 'epoch': 1.0})

## failed idea: have the model go through a timewise curriculum of the data
The idea was that if the training didn't respect the timeseries nature of the data, then the model could "memorize" parts of the data and use that to predict past data better (which wouldn't be good at test time). Seemingly this isn't an issue as the model performs better on a validation set that does come from the future.

In [3]:
class TraderTrainer(Trainer):

    def get_train_dataloader(self) -> DataLoader:
        """
        Returns the training :class:`~torch.utils.data.DataLoader`.

        Will use no sampler if :obj:`self.train_dataset` does not implement :obj:`__len__`, a random sampler (adapted
        to distributed training if necessary) otherwise.

        Subclass and override this method if you want to inject some custom behavior.
        """
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")
        train_sampler = self._get_train_sampler()

        return DataLoader(
            self.train_dataset,
            batch_size=self.args.train_batch_size,
#             shuffle=False, # TO STOP OVERFITTING
            sampler=train_sampler,
            collate_fn=self.data_collator,
            drop_last=self.args.dataloader_drop_last,
            num_workers=self.args.dataloader_num_workers,
        )

## make sure unshuffled split maintains order

In [16]:
foo = Dataset.from_dict({"input": list(range(100))})
split = foo.train_test_split(.1, shuffle = False)
valid_test = split['test'].train_test_split(.5, shuffle = False)
foo = DatasetDict({
    'train': split['train'],
    'validation': valid_test['train'],
    'test': valid_test['test']
})

In [20]:
foo['validation']['input'], foo['test']['input']

([90, 91, 92, 93, 94], [95, 96, 97, 98, 99])

## quick timing check

In [ ]:
model = GPT2Trader(config).cuda()

In [72]:
%%timeit
fake_data = torch.randn(4, 391, 256)
fake_data = fake_data.cuda()
model(fake_data)
cpu = fake_data.cpu()

28.2 ms ± 2.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [73]:
model = GPT2Trader(config).cpu()

Using 9 layers


In [74]:
%%timeit
fake_data = torch.randn(4, 391, 256)
model(fake_data)

748 ms ± 82.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
